# Language Identification with Impresso Pipelines Package

## What is this notebook about?


This notebook demonstrates how to use the language identification subpackage `langident` from the Impresso Python package. This subpackage allows to detect the language of a given text and provides simple diagnostic features to analyze the decection process.

---
## What will you learn?

In this notebook, you will:

* Understand how to use the `langident` subpackage from the Impresso Pipelines package for language detection.

* Learn how to classify text into different languages using a simple pipeline.

* Explore diagnostic features to analyze language predictions.

* Identify common challenges in language detection, such as handling short texts or ambiguous words.
*   Experiment with multilingual and edge-case scenarios to observe model behavior.

This hands-on guide will provide you with practical insights into language identification and its limitations. ​

---
## Prerequisites

First, you should install Impresso Pipelines package:

In [22]:
%pip install impresso_pipelines[langident]

---
## Workflow

### Basic usage

Start by importing the necessary module from Impresso Pipelines package

In [6]:
from impresso_pipelines.langident import LangIdentPipeline
lang_pipeline = LangIdentPipeline()

Once you initialize the pipeline, you can simply provide the text you'd like to classify. This example demonstrates the use of German text.

In [7]:
de_text = "Ein kleiner Hund namens Max lebte in einem ruhigen Dorf."

In [8]:
lang_pipeline(de_text)

{'language': 'de', 'score': 1.0}

The default output of the pipeline is a dictionary containing the top predicted language and its corresponding score (expressed as a probability). The score is rounded to three decimal places for better readability. Please note that the probabilities for all supported languages add up to 1 (by default, only the top language is returned).

As shown, the pipeline uses the language identification model to correctly classify this text as German, with a rounded probability of 100%.

### Advanced usage

When using the pipeline with text, you can additionally specify two parameters: `diagnostics` and `model_id`.


*   `diagnostics`: A boolean value. If set to True, it returns not only the top predicted language but also all languages that the model can detect, along with their corresponding scores.

*   `model_id`: A boolean value. If set to True, it returns the name of the model used to identify the language of the text.

Here we skip the part of module importing and initialization as it was done above.

In [9]:
# Using text example from before
lang_pipeline(de_text, diagnostics=True)

{'language': 'de',
 'score': 1.0,
 'diagnostics': {'languages': [{'language': 'de', 'score': 1.0},
   {'language': 'fr', 'score': 0.0},
   {'language': 'it', 'score': 0.0},
   {'language': 'lb', 'score': 0.0},
   {'language': 'en', 'score': 0.0}]}}

As shown, it returns a `language_dict` containing a list of all supported languages and their corresponding scores. Since the text is purely in German, all other scores are 0.0.

Below is an example of using `model_id` with and without `diagnostics`.

In [10]:
lang_pipeline(de_text, model_id=True)

{'language': 'de', 'score': 1.0, 'model_id': 'langident-v1.0.0.bin'}

In [11]:
lang_pipeline(de_text, model_id=True, diagnostics=True)

{'language': 'de',
 'score': 1.0,
 'diagnostics': {'languages': [{'language': 'de', 'score': 1.0},
   {'language': 'fr', 'score': 0.0},
   {'language': 'it', 'score': 0.0},
   {'language': 'lb', 'score': 0.0},
   {'language': 'en', 'score': 0.0}]},
 'model_id': 'langident-v1.0.0.bin'}

In both cases, we can see an additional key, `model_name`, which stores the name of the language identification model used by the pipeline.

### Mixed text example

In [12]:
mixed_text = "Max marchait doucement. Der vento soffiait fort, aber la strada restait vide."


In [13]:
lang_pipeline(mixed_text, diagnostics=True)

{'language': 'fr',
 'score': 0.55,
 'diagnostics': {'languages': [{'language': 'fr', 'score': 0.55},
   {'language': 'de', 'score': 0.31},
   {'language': 'it', 'score': 0.1},
   {'language': 'en', 'score': 0.04},
   {'language': 'lb', 'score': 0.01}]}}

As shown, this time the model clearly detects some French and even Italian, but French remains the top predicted language, with German as the second most likely.

### Advanced Initialization

By default, the pipeline automatically selects the most recent language identification model from the Impresso HF repository: `impresso-project/impresso-floret-langident`.

However, the module initialization allows you to pass additional arguments to use a specific model instead of the default one. These arguments include `model_id`, `repo_id`, and `revision`.

*   `model_id`: Specifies the name of the model.
*   `repo_id`: Specifies the repository where the model is located.

*   `revision`: Specifies the branch name of the repository.

By providing all three, you can force the pipeline to use the language model you have specified.

In [14]:
from impresso_pipelines.langident import LangIdentPipeline # There's no need to import the module again if it has already been imported
lang_pipeline = LangIdentPipeline(model_id="langident-v1.0.0.bin", repo_id="impresso-project/impresso-floret-langident", revision="main")

In [15]:
# Using text example from before
lang_pipeline(de_text)

{'language': 'de', 'score': 1.0}

Once again, we see the same pipeline output as before.

### Common Pitfalls in Language Detection

In [16]:
short_fr_text = "Je mange."

In [17]:
short_de_text = "Der Computer auf dem Tisch funktioniert gut."

In [18]:
short_de_text_with_unusual_name = "Gleb geht ins Büro."

In [19]:
# Example 1: Very short sentence
lang_pipeline(short_fr_text, diagnostics=True)

{'language': 'fr',
 'score': 0.67,
 'diagnostics': {'languages': [{'language': 'fr', 'score': 0.67},
   {'language': 'lb', 'score': 0.33},
   {'language': 'de', 'score': 0.0},
   {'language': 'it', 'score': 0.0},
   {'language': 'en', 'score': 0.0}]}}

In [20]:
# Exaple 2: Not language specific sentence
lang_pipeline(short_de_text, diagnostics=True)

{'language': 'de',
 'score': 0.61,
 'diagnostics': {'languages': [{'language': 'de', 'score': 0.61},
   {'language': 'lb', 'score': 0.38},
   {'language': 'it', 'score': 0.01},
   {'language': 'en', 'score': 0.0},
   {'language': 'fr', 'score': 0.0}]}}

In [21]:
# Example 3: Short sentence and unsual name
lang_pipeline(short_de_text_with_unusual_name, diagnostics=True)

{'language': 'lb',
 'score': 0.52,
 'diagnostics': {'languages': [{'language': 'lb', 'score': 0.52},
   {'language': 'de', 'score': 0.43},
   {'language': 'en', 'score': 0.04},
   {'language': 'it', 'score': 0.01},
   {'language': 'fr', 'score': 0.0}]}}

As demonstrated, this pipeline struggles to accurately detect the language when the text is too short. This challenge becomes even more pronounced when the words used are not strongly tied to a specific language. Additionally, the model encounters difficulties with short sentences that contain uncommon names. In general, the longer the text sample, the higher the detection accuracy.

As shown above, despite low confidence scores, the pipeline correctly predicts the language in the first two cases (a short French text and a non-language-specific German text). However, in the third example—where the sentence is both short and includes an unusual, non-German name—the model makes an incorrect prediction.

This example highlights the importance of longer, more language-distinctive sentences for achieving higher accuracy and confidence in language classification.

---
## Conclusion

### Summary

This notebook provides a step-by-step guide on using the `langident` subpackage from the Impresso Python package for language detection. It begins with an introduction to the package and instructions on installing the necessary dependencies.

The workflow section covers:

* Basic Usage: Initializing the language identification pipeline and classifying single-language texts.
* Advanced Usage: Exploring additional pipeline features, such as retrieving full probability distributions for multiple languages.
* Handling Challenging Cases: Analyzing model limitations when dealing with short or ambiguous texts, multilingual content, and names that may not be strongly language-specific.

### Next steps

You might also be interested in a follow-up notebook on the Impresso Pipeline package, [OCRQA](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/ocrqa_pipeline_demo.ipynb), which utilizes the `langident` language detection.

Additionally, you can find the repository for the **Impresso Pipelines** package [here](https://github.com/impresso/impresso-pipelines/tree/mallet_pipeline).




---
## Project and License info

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an interdisciplinary research project that aims to develop and consolidate tools for processing and exploring large collections of media archives across modalities, time, languages and national borders. The first project (2017-2021) was funded by the Swiss National Science Foundation under grant No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027) by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585) and the Luxembourg National Research Fund under grant No. 17498891.

### Copyright

Copyright (C) 2024 The Impresso team.

### License

This program is provided as open source under the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE) v3 or later.

---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>


